In [1]:
import torch
import time
import gc

from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    LlamaForCausalLM,
)

from peft import LoraConfig, PeftModel, get_peft_model
import time
from transformers import logging
logging.set_verbosity_error()

In [2]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"

In [3]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True,  
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_use_double_quant=True,)

In [4]:
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["gate_proj", "down_proj", "up_proj"]
)

In [5]:
def test(model, runs=10):
    inp = tokenizer.encode("This is a long story about Alise in wonderland.\n", return_tensors="pt").to("cuda")
    len_inp = len(inp[0])
    generated_tokens = 0
    with torch.inference_mode():
        # load model
        _ = model.generate(inp, max_new_tokens=1000, do_sample=False, num_beams=1, temperature=0, top_p=1.0)
        t_start = time.perf_counter()
        for _ in range(runs):
            out = model.generate(inp, max_new_tokens=1000, do_sample=False, num_beams=1, temperature=0, top_p=1.0)
            generated_tokens += len(out[0]) - len_inp
        t_total = time.perf_counter() - t_start
    del(inp)
    torch.cuda.empty_cache()
    gc.collect()
    return {"time_total": t_total, "generated_tokens": generated_tokens, "tokens/sec": generated_tokens / t_total}

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
#Create a new token and add it to the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

---
## Load models

In [7]:
n_runs = 10

In [8]:
model_original = LlamaForCausalLM.from_pretrained(
            model_name,
            quantization_config=None,
            attn_implementation="sdpa",
            device_map=("cuda"),
            torch_dtype=torch.bfloat16)

In [9]:
res_1 = test(model_original, n_runs)
print("Original:\n", res_1)

/home/otniel/miniconda3/envs/dl/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Original:
 {'time_total': 103.2377604819776, 'generated_tokens': 8660, 'tokens/sec': 83.88403583698226}


In [15]:
model_w_adapter = get_peft_model(model_original, peft_config)

In [11]:
res_2 = test(model_w_adapter, n_runs)
print("With Adapter:\n", res_2)

With Adapter:
 {'time_total': 156.78170123900054, 'generated_tokens': 8660, 'tokens/sec': 55.23603795317004}


In [18]:
model_merged = model_w_adapter.merge_and_unload()

In [13]:
res_3 = test(model_merged, n_runs)
print("Merged:\n", res_3)

Merged:
 {'time_total': 102.39856893397518, 'generated_tokens': 8660, 'tokens/sec': 84.57149440812807}


---